# **Signal To Spike Conversion** - Analysis

This notebook conducts an analysis of the Inter-Spike Interval in the UP and DN spike trains to determine the optimal synaptic time constants

### Check WD (change if necessary) and file loading

In [74]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/signal_to_spike" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/signal_to_spike"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike


### Declare the `INPUT_FOLDER` and `RESULTS_FOLDER`

In [75]:
# CAREFUL WITH THIS FOLDER TO NOT OVERWRITE THE FILES
DATASET_FILENAME = "custom_subset_90-119_segment500_200"
INPUT_FOLDER = f"results/{DATASET_FILENAME}"
RESULTS_FOLDER = "analysis/{DATASET_FILENAME}"

## Load the UP and DN spike trains in the Ripple and FR Band

In [76]:
import numpy as np
import math
from utils.io import preview_np_array
from utils.input import read_spike_events, MarkerType, band_to_file_name

thresh_up = 1.8974
thresh_down = -thresh_up

In [77]:
ripple_band_filename = band_to_file_name(MarkerType.RIPPLE)
fr_band_filename = band_to_file_name(MarkerType.FAST_RIPPLE)

# Call the function to read the spike events
ripple_up_spikes_file_path = f"{INPUT_FOLDER}/{ripple_band_filename}_up_spike_train_{thresh_up}.csv"
ripple_down_spikes_file_path = f"{INPUT_FOLDER}/{ripple_band_filename}_down_spike_train_{thresh_down}.csv"

ripple_up_spike_train = read_spike_events(ripple_up_spikes_file_path)
ripple_down_spike_train = read_spike_events(ripple_down_spikes_file_path)


preview_np_array(ripple_up_spike_train, "ripple_up_spike_train", edge_items=3)
preview_np_array(ripple_down_spike_train, "ripple_down_spike_train", edge_items=3)

ripple_up_spike_train Shape: (10893, 2).
Preview: [[ 4.19921875e+01 -1.00000000e+00]
 [ 5.12695312e+01 -1.00000000e+00]
 [ 5.90820312e+01 -1.00000000e+00]
 ...
 [ 1.19844727e+05 -1.00000000e+00]
 [ 1.19854492e+05 -1.00000000e+00]
 [ 1.19996582e+05 -1.00000000e+00]]
ripple_down_spike_train Shape: (10905, 2).
Preview: [[ 4.44335938e+01 -1.00000000e+00]
 [ 5.46875000e+01 -1.00000000e+00]
 [ 5.61523438e+01 -1.00000000e+00]
 ...
 [ 1.19842285e+05 -1.00000000e+00]
 [ 1.19850586e+05 -1.00000000e+00]
 [ 1.19993164e+05 -1.00000000e+00]]


In [78]:
fr_up_spikes_file_path = f"{INPUT_FOLDER}/{fr_band_filename}_up_spike_train_{thresh_up}.csv"
fr_down_spikes_file_path = f"{INPUT_FOLDER}/{fr_band_filename}_down_spike_train_{thresh_down}.csv"

fr_up_spike_train = read_spike_events(fr_up_spikes_file_path)
fr_down_spike_train = read_spike_events(fr_down_spikes_file_path)

preview_np_array(fr_up_spike_train, "fr_up_spike_train", edge_items=3)
preview_np_array(fr_down_spike_train, "fr_down_spike_train", edge_items=3)

fr_up_spike_train Shape: (17319, 2).
Preview: [[ 1.31835938e+01 -1.00000000e+00]
 [ 1.51367188e+01 -1.00000000e+00]
 [ 4.49218750e+01 -1.00000000e+00]
 ...
 [ 1.19993652e+05 -1.00000000e+00]
 [ 1.19995605e+05 -1.00000000e+00]
 [ 1.19998047e+05 -1.00000000e+00]]
fr_down_spike_train Shape: (17166, 2).
Preview: [[ 1.41601562e+01 -1.00000000e+00]
 [ 4.39453125e+01 -1.00000000e+00]
 [ 5.27343750e+01 -1.00000000e+00]
 ...
 [ 1.19994629e+05 -1.00000000e+00]
 [ 1.19996582e+05 -1.00000000e+00]
 [ 1.19999023e+05 -1.00000000e+00]]


### Load the Annotated Data
The annotated data is needed to separate the intervals of HFO from the normal activity to calculate the Inter-Spike Interval (ISI)

In [79]:
# load the npy object
ripple_ground_truth = np.load(f"{INPUT_FOLDER}/{ripple_band_filename}_ground_truth.npy")
fr_ground_truth = np.load(f"{INPUT_FOLDER}/{fr_band_filename}_ground_truth.npy")

preview_np_array(ripple_ground_truth, "ripple_ground_truth", edge_items=3)
preview_np_array(fr_ground_truth, "fr_ground_truth", edge_items=3)

ripple_ground_truth Shape: (222,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Spike+Ripple',   3521.  , 0.) ... ('Spike+Ripple', 116216.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Ripple', 119000.  , 0.)]
fr_ground_truth Shape: (199,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Fast-Ripple',   3770.02, 0.) ... ('Fast-Ripple', 116096.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Fast-Ripple', 119000.  , 0.)]


## Iterate the UP and DN Spike Trains to Calculate the Inter-Spike Interval (ISI) in the HFO and Normal Activity

Define the lists to store:
- `Ripple_HFO_UP` - ISI of the UP train in the Ripple band during an HFO event 
- `Ripple_HFO_DN` - ISI of the DN train in the Ripple band during an HFO event
- `Ripple_Noise_UP` - ISI of the UP train in the Ripple band during normal activity
- `Ripple_Noise_DN` - ISI of the DN train in the Ripple band during normal activity

- `FR_HFO_UP` - ISI of the UP train in the Fast Ripple band during an HFO event
- `FR_HFO_DN` - ISI of the DN train in the Fast Ripple band during an HFO event
- `FR_Noise_UP` - ISI of the UP train in the Fast Ripple band during normal activity
- `FR_Noise_DN` - ISI of the DN train in the Fast Ripple band during normal activity

## Let's analyze the Ripple Band first

In [80]:
isi_ripple_hfo_up = []
isi_ripple_hfo_down = []
isi_ripple_noise_up = []
isi_ripple_noise_down = []

#### Ripple Band - UP Train

In [81]:
from utils.input import RIPPLE_CONFIDENCE_WINDOW, FR_CONFIDENCE_WINDOW, BOTH_CONFIDENCE_WINDOW

# Iterate the Ripple UP Spike Train
curr_ripple_gt_idx = 0  # Current index of the ripple ground truth
prev_spike_time = ripple_up_spike_train[0][0]

for spike_idx in range(1, len(ripple_up_spike_train), 1):
    spike_time = ripple_up_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_ripple_gt_idx < len(ripple_ground_truth) and spike_time > ripple_ground_truth[curr_ripple_gt_idx][1] + RIPPLE_CONFIDENCE_WINDOW:
        curr_ripple_gt_idx += 1
    
    # Check if the current spike time is within the ripple ground truth
    if curr_ripple_gt_idx < len(ripple_ground_truth):
        curr_gt_start = ripple_ground_truth[curr_ripple_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + RIPPLE_CONFIDENCE_WINDOW:
            # The spike is within the ripple ground truth
            isi_ripple_hfo_up.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the ripple ground truth
    isi_ripple_noise_up.append(isi)

#### Ripple Band - DOWN Train

In [82]:
# Iterate the Ripple DOWN Spike Train
curr_ripple_gt_idx = 0  # Current index of the ripple ground truth
prev_spike_time = ripple_down_spike_train[0][0]

for spike_idx in range(1, len(ripple_down_spike_train), 1):
    spike_time = ripple_down_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_ripple_gt_idx < len(ripple_ground_truth) and spike_time > ripple_ground_truth[curr_ripple_gt_idx][1] + RIPPLE_CONFIDENCE_WINDOW:
        curr_ripple_gt_idx += 1
    
    # Check if the current spike time is within the ripple ground truth
    if curr_ripple_gt_idx < len(ripple_ground_truth):
        curr_gt_start = ripple_ground_truth[curr_ripple_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + RIPPLE_CONFIDENCE_WINDOW:
            # The spike is within the ripple ground truth
            isi_ripple_hfo_down.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the ripple ground truth
    isi_ripple_noise_down.append(isi)

### Validate the number of calculated ISIs
Validate that:
- `len(isi_ripple_hfo_up)` + `len(isi_ripple_noise_up)` = `len(ripple_up)` - 1 (first spike does not have an ISI)
- `len(isi_ripple_hfo_down_)` + `len(isi_ripple_noise_down_)` = `len(ripple_down_)` - 1 (first spike does not have an ISI)

In [83]:
#  Validate that len(isi_ripple_hfo_up) + len(isi_ripple_noise_up) = len(ripple_up)
print(f"Length isi_ripple_hfo_up: {len(isi_ripple_hfo_up)}")
print(f"Length isi_ripple_noise_up: {len(isi_ripple_noise_up)}")
print(f"Sum: {len(isi_ripple_hfo_up) + len(isi_ripple_noise_up)}")
print(f"Length ripple_up: {len(ripple_up_spike_train)}")

Length isi_ripple_hfo_up: 5462
Length isi_ripple_noise_up: 5430
Sum: 10892
Length ripple_up: 10893


In [84]:
#  Validate that len(isi_ripple_hfo_down) + len(isi_ripple_noise_down) = len(ripple_down)
print(f"Length isi_ripple_hfo_down: {len(isi_ripple_hfo_down)}")
print(f"Length isi_ripple_noise_down: {len(isi_ripple_noise_down)}")
print(f"Sum: {len(isi_ripple_hfo_down) + len(isi_ripple_noise_down)}")
print(f"Length ripple_down: {len(ripple_down_spike_train)}")

Length isi_ripple_hfo_down: 5421
Length isi_ripple_noise_down: 5483
Sum: 10904
Length ripple_down: 10905


## Let's analyze the Fast Ripple Band next

In [85]:
isi_fr_hfo_up = []
isi_fr_hfo_down = []
isi_fr_noise_up = []
isi_fr_noise_down = []

#### Fast Ripple Band - UP Train

In [86]:
# Iterate the FR UP Spike Train
curr_fr_gt_idx = 0  # Current index of the fast ripple ground truth
prev_spike_time = fr_up_spike_train[0][0]

for spike_idx in range(1, len(fr_up_spike_train), 1):
    spike_time = fr_up_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the fast ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_fr_gt_idx < len(fr_ground_truth) and spike_time > fr_ground_truth[curr_fr_gt_idx][1] + FR_CONFIDENCE_WINDOW:
        curr_fr_gt_idx += 1
    
    # Check if the current spike time is within the fast ripple ground truth
    if curr_fr_gt_idx < len(fr_ground_truth):
        curr_gt_start = fr_ground_truth[curr_fr_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + FR_CONFIDENCE_WINDOW:
            # The spike is within the fast ripple ground truth
            isi_fr_hfo_up.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the fast ripple ground truth
    isi_fr_noise_up.append(isi)

#### Fast Ripple Band - DN Train

In [87]:
# Iterate the FR UP Spike Train
curr_fr_gt_idx = 0  # Current index of the fast ripple ground truth
prev_spike_time = fr_down_spike_train[0][0]

for spike_idx in range(1, len(fr_down_spike_train), 1):
    spike_time = fr_down_spike_train[spike_idx][0]
    isi = spike_time - prev_spike_time
    # print("spike_time: ", spike_time, "prev_spike_time: ", prev_spike_time, "isi: ", isi)

    # Update the previous spike time
    prev_spike_time = spike_time

    # Check if the current spike time is within the fast ripple ground truth
    # Skip the ground truth that is before the current spike time
    while curr_fr_gt_idx < len(fr_ground_truth) and spike_time > fr_ground_truth[curr_fr_gt_idx][1] + FR_CONFIDENCE_WINDOW:
        curr_fr_gt_idx += 1
    
    # Check if the current spike time is within the fast ripple ground truth
    if curr_fr_gt_idx < len(fr_ground_truth):
        curr_gt_start = fr_ground_truth[curr_fr_gt_idx][1]
        if spike_time >= curr_gt_start and spike_time <= curr_gt_start + FR_CONFIDENCE_WINDOW:
            # The spike is within the fast ripple ground truth
            isi_fr_hfo_down.append(isi)

            # Go to the next spike
            continue
    
    # The spike is not within the fast ripple ground truth
    isi_fr_noise_down.append(isi)

### Validate the number of calculated ISIs
Validate that:
- `len(isi_fr_hfo_up)` + `len(isi_fr_noise_up)` = `len(fr_up)` - 1 (first spike does not have an ISI)
- `len(isi_fr_hfo_down)` + `len(isi_fr_noise_down)` = `len(fr_down)` - 1 (first spike does not have an ISI)

In [88]:
#  Validate that len(isi_fr_hfo_up) + len(isi_fr_noise_up) = len(ripple_up)
print(f"Length isi_fr_hfo_up: {len(isi_fr_hfo_up)}")
print(f"Length isi_fr_noise_up: {len(isi_fr_noise_up)}")
print(f"Sum: {len(isi_fr_hfo_up) + len(isi_fr_noise_up)}")
print(f"Length fr_up: {len(fr_up_spike_train)}")

Length isi_fr_hfo_up: 4288
Length isi_fr_noise_up: 13030
Sum: 17318
Length fr_up: 17319


In [89]:
#  Validate that len(isi_fr_hfo_down) + len(isi_fr_noise_down) = len(ripple_down)
print(f"Length isi_fr_hfo_down: {len(isi_fr_hfo_down)}")
print(f"Length isi_fr_noise_down: {len(isi_fr_noise_down)}")
print(f"Sum: {len(isi_fr_hfo_down) + len(isi_fr_noise_down)}")
print(f"Length fr_down: {len(fr_down_spike_train)}")

Length isi_fr_hfo_down: 4261
Length isi_fr_noise_down: 12904
Sum: 17165
Length fr_down: 17166


--- 

## Calculate metrics of the ISIs and Show the Results

In [90]:
# Ripple band ISIs
# UP
mean_ripple_hfo_up = np.mean(isi_ripple_hfo_up)
median_ripple_hfo_up = np.median(isi_ripple_hfo_up)
std_ripple_hfo_up = np.std(isi_ripple_hfo_up)
mean_ripple_noise_up = np.mean(isi_ripple_noise_up)
median_ripple_noise_up = np.median(isi_ripple_noise_up)
std_ripple_noise_up = np.std(isi_ripple_noise_up)

# DOWN
mean_ripple_hfo_down = np.mean(isi_ripple_hfo_down)
median_ripple_hfo_down = np.median(isi_ripple_hfo_down)
std_ripple_hfo_down = np.std(isi_ripple_hfo_down)
mean_ripple_noise_down = np.mean(isi_ripple_noise_down)
median_ripple_noise_down = np.median(isi_ripple_noise_down)
std_ripple_noise_down = np.std(isi_ripple_noise_down)

In [91]:
# Fast Ripple band ISIs
# UP
mean_fr_hfo_up = np.mean(isi_fr_hfo_up)
median_fr_hfo_up = np.median(isi_fr_hfo_up)
std_fr_hfo_up = np.std(isi_fr_hfo_up)
mean_fr_noise_up = np.mean(isi_fr_noise_up)
median_fr_noise_up = np.median(isi_fr_noise_up)
std_fr_noise_up = np.std(isi_fr_noise_up)

# DOWN
mean_fr_hfo_down = np.mean(isi_fr_hfo_down)
median_fr_hfo_down = np.median(isi_fr_hfo_down)
std_fr_hfo_down = np.std(isi_fr_hfo_down)
mean_fr_noise_down = np.mean(isi_fr_noise_down)
median_fr_noise_down = np.median(isi_fr_noise_down)
std_fr_noise_down = np.std(isi_fr_noise_down)

### Print the Metrics of the ISIs

#### Ripple Band

In [92]:
# Metrics of the ISIs in the Ripple Band
print("Ripple UP")
print("HFO ISI (ms)")
print(f"Mean: {mean_ripple_hfo_up}, Median: {median_ripple_hfo_up}, STD: {std_ripple_hfo_up}")
print("Noise ISI (ms)")
print(f"Mean: {mean_ripple_noise_up}, Median: {median_ripple_noise_up}, STD: {std_ripple_noise_up}")

print("=================================")
print("Ripple DOWN")
print("HFO ISI (ms)")
print(f"Mean: {mean_ripple_hfo_down}, Median: {median_ripple_hfo_down}, STD: {std_ripple_hfo_down}")
print("Noise ISI (ms)")
print(f"Mean: {mean_ripple_noise_down}, Median: {median_ripple_noise_down}, STD: {std_ripple_noise_down}")

Ripple UP
HFO ISI (ms)
Mean: 4.099703062065178, Median: 0.9765625, STD: 7.691148414627944
Noise ISI (ms)
Mean: 17.967221311003684, Median: 10.7421875, STD: 26.94784945331581
Ripple DOWN
HFO ISI (ms)
Mean: 4.165928074847814, Median: 0.9765625, STD: 7.84948772741277
Noise ISI (ms)
Mean: 17.75765719040671, Median: 10.7421875, STD: 26.784767260572362


#### Fast Ripple Band

In [ ]:
# Metrics of the ISIs in the FR Band
print("FR UP")
print("HFO ISI (ms)")
print(f"Mean: {mean_fr_hfo_up}, Median: {median_fr_hfo_up}, STD: {std_fr_hfo_up}")
print("Noise ISI (ms)")
print(f"Mean: {mean_fr_noise_up}, Median: {median_fr_noise_up}, STD: {std_fr_noise_up}")

print("=================================")
print("FR DOWN")
print("HFO ISI (ms)")
print(f"Mean: {mean_fr_hfo_down}, Median: {median_fr_hfo_down}, STD: {std_fr_hfo_down}")
print("Noise ISI (ms)")
print(f"Mean: {mean_fr_noise_down}, Median: {median_fr_noise_down}, STD: {std_fr_noise_down}")